In [1]:
# Here I choose Gradient Boosting model as my ML model. i prefer to go with it. 
# i do not try multiple models here.

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [26]:
df = pd.read_csv('./datasets/preprocessed.csv')

In [27]:
df.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Month,Day
0,22,8.694688,13.589273,0.6,2.263869,57.554920,0,8.870193,0,1,...,318.255596,1.375390,8.0,0.0,10.549671,13.043088,0,0,12,1
1,22,5.301024,14.668806,0.0,2.406140,69.201128,1,8.870193,4,2,...,319.018654,1.375395,7.0,1.0,10.705502,14.278149,0,0,12,2
2,22,8.423478,14.960437,0.0,2.496163,59.559201,2,9.074396,0,2,...,318.229277,1.375401,8.0,2.0,12.642979,13.737467,0,0,12,3
3,22,6.355600,16.068119,0.0,2.589230,77.023050,3,6.419909,12,14,...,320.859017,1.375428,0.0,5.0,11.170317,15.347521,0,0,12,4
4,22,10.860881,18.099591,1.0,2.819463,88.580965,0,8.553172,8,11,...,319.071265,1.375382,7.0,8.0,11.015816,16.877070,0,0,12,5


In [19]:
df.shape

(145460, 24)

In [21]:
df.isnull().mean()*100

Location         0.000000
MinTemp          2.381411
MaxTemp          0.000000
Rainfall         0.000000
Evaporation      0.800907
Sunshine         0.000000
WindGustDir      0.000000
WindGustSpeed    0.000000
WindDir9am       0.000000
WindDir3pm       0.000000
WindSpeed9am     0.000000
WindSpeed3pm     0.000000
Humidity9am      0.000000
Humidity3pm      3.098446
Pressure9am      0.000000
Pressure3pm      0.000000
Cloud9am         0.000000
Cloud3pm         0.000000
Temp9am          0.304551
Temp3pm          0.000000
RainToday        0.000000
RainTomorrow     0.000000
Month            0.000000
Day              0.000000
dtype: float64

In [6]:
y = df['RainTomorrow']
X = df.drop(['RainTomorrow'],axis=1)

In [7]:
X.columns

Index(['Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'Month', 'Day'],
      dtype='object')

In [8]:
y


0         0
1         0
2         0
3         0
4         0
         ..
145455    0
145456    0
145457    0
145458    0
145459    0
Name: RainTomorrow, Length: 145460, dtype: int64

In [9]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

os=RandomOverSampler(0.9)
X_ns,y_ns=os.fit_sample(X,y)
print("The number of classes before fit {}".format(Counter(y)))
print("The number of classes after fit {}".format(Counter(y_ns)))

X = X_ns
y = y_ns

C:\Users\Acer\anaconda3\lib\site-packages\imblearn\utils\_validation.py:638: FutureWarning: Pass sampling_strategy=0.9 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning)


The number of classes before fit Counter({0: 113583, 1: 31877})
The number of classes after fit Counter({0: 113583, 1: 102224})


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [11]:
from sklearn.ensemble import GradientBoostingClassifier

In [12]:
gb = GradientBoostingClassifier()

In [13]:
gb.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'deviance',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [14]:
grid={
    'learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001], 
    'n_estimators':[x for x in range(100,1000,2)],
    'max_depth':[2,3,4,5,6,7],
    'min_samples_split':[2,4,6,8,10,20,40,60,100], 
    'min_samples_leaf':[1,3,5,7,9],
    'max_features':[2,3,4,5,6,7],
    'subsample':[0.7,0.75,0.8,0.85,0.9,0.95,1]
}

In [15]:
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=10, shuffle=True)

In [16]:
from sklearn.model_selection import RandomizedSearchCV
gb_random = RandomizedSearchCV(estimator=gb, param_distributions=grid, n_jobs=-1,n_iter = 50, cv=cv, scoring='accuracy',error_score=0,return_train_score=True,verbose=2)

In [17]:
gb_random.fit(X_train,y_train)

Fitting 10 folds for each of 50 candidates, totalling 500 fits


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
best_model = gb_random.best_estimator_

In [ ]:
best_model

In [ ]:
gb_random.best_params_

In [ ]:
gb_random.best_score_

In [ ]:
y_pred = best_model.predict(X_test)

In [ ]:
err  =(y_pred- y_test)

In [ ]:
err.value_counts().plot(kind='bar',color='purple')

In [ ]:
from sklearn import metrics
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
indices=list(X_train)
feat_imp = pd.Series(best_model.feature_importances_, indices).sort_values(ascending=False)
feat_imp.plot(kind='bar', title='Importance of Features')
plt.ylabel('Feature Importance Score')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
print("accuracy is {}".format(best_model.score(X_test, y_test)))

In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [ ]:
baseline_roc_auc = roc_auc_score(y_test, best_model.predict(X_test))
fprB, tprB, thresholdsB = roc_curve(y_test, best_model.predict_proba(X_test)[:,1])

In [ ]:
plt.figure()
plt.plot(fprB, tprB, label='GB(area = %0.2f)' % baseline_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# ROC - AUC simple code
#metrics.plot_roc_curve(knn, X_test, y_test)
#metrics.roc_auc_score(y_test, y_pred4, average=None)

In [ ]:
### Predict test_y values and probabilities based on random forest model
probs_y=best_model.predict_proba(X_test) 
  # probs_y is a 2-D array of probability of being labeled as 0 (first column of array) vs 1 (2nd column in array)

from sklearn.metrics import precision_recall_curve
precision, recall, thresholds = precision_recall_curve(y_test, probs_y[:, 1]) 
   #retrieve probability of being 1(in second column of probs_y)
pr_auc = metrics.auc(recall, precision)

plt.title("Precision-Recall vs Threshold Chart")
plt.plot(thresholds, precision[: -1], "b--", label="Precision")
plt.plot(thresholds, recall[: -1], "r--", label="Recall")
plt.ylabel("Precision, Recall")
plt.xlabel("Threshold")
plt.legend(loc="lower left")
plt.ylim([0,1])

In [ ]:
import pickle
#open a file, where you ant to store the data
file = open('gboost.pkl', 'wb')
# dump information to that file
pickle.dump(best_model, file)
file.close()